In [ ]:
#generating tifs for quiver timelapse

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas

# Create a directory to save the TIFF images if it doesn't exist
output_dir = "quiver_tifs"
os.makedirs(output_dir, exist_ok=True)

# Define the range of frames you want to process
start_frame = first frame
end_frame = last frame

# Loop through each frame
for frame_number in range(start_frame, end_frame + 1):
    # Construct file paths for x and y velocity components
    file_x = f'Velocity_Map_X_Frame_{frame_number}.xlsx'
    file_y = f'Velocity_Map_Y_Frame_{frame_number}.xlsx'

    # Read velocity data from Excel files
    df_x = pd.read_excel(file_x, header=None)
    df_y = pd.read_excel(file_y, header=None)

    # Convert to velocity in um/min
    vxt = df_x.values * 5.86
    vyt = df_y.values * 5.86

    # Invert y-velocity to match positive y-direction upwards
    vyt = -vyt

    # Create figure and axes for quiver plot
    fig, ax = plt.subplots(figsize=(8, 6))
    q = ax.quiver(vxt, vyt, scale=50, pivot='middle', color='green', width=0.002, headwidth=3, headlength=4)

    # Set plot limits based on data dimensions
    ax.set_xlim(0, vxt.shape[1])
    ax.set_ylim(0, vyt.shape[0])  # Ensure y-axis starts from 0 upwards

    # Save the plot as a TIFF file
    fig.canvas = FigureCanvas(fig)
    fig.savefig(f'Quiver_Frame_{frame_number}.tif', bbox_inches='tight')

    # Close the figure to release memory
    plt.close(fig)

print("All quiver plots saved successfully.")

In [ ]:
#moving quiver files to folder

import os
import shutil

# Source directory containing the TIFF files
source_dir = '.'

# Destination directory for the selected TIFF files
destination_dir = 'quiver_tifs'

# Create the destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# Loop through the specified range of frames and move the files
for frame_number in range(40, 411):
    file_name = f'Quiver_Frame_{frame_number}.tif'
    source_path = os.path.join(source_dir, file_name)
    destination_path = os.path.join(destination_dir, file_name)
    
    # Check if the file exists before attempting to move it
    if os.path.exists(source_path):
        shutil.move(source_path, destination_path)
        print(f'Moved: {file_name}')
    else:
        print(f'File not found: {file_name}')

In [ ]:
#converting quiver tifs to avi for timelapse

import cv2
import os
import re

# Directory containing the quiver plot TIFF files
input_dir = "quiver_tifs"

# Output video file name
output_avi = "quiver_timelapse_mjpg.avi"

# Frame rate for the video
frame_rate = 5  # frames per second (adjust as needed)

# Get a list of all TIFF files in the directory, sorted by filename (numeric order)
tiff_files = sorted(os.listdir(input_dir), key=lambda x: int(re.findall(r'\d+', x)[0]))

# Check if there are TIFF files to process
if not tiff_files:
    print("No TIFF files found in the directory.")
    exit()

# Read the first TIFF file to get the frame size
first_frame = cv2.imread(os.path.join(input_dir, tiff_files[0]))
height, width, layers = first_frame.shape

# Define the codec and create a VideoWriter object for AVI
fourcc_avi = cv2.VideoWriter_fourcc(*'MJPG')  # Using MJPG codec for AVI
video_writer_avi = cv2.VideoWriter(output_avi, fourcc_avi, frame_rate, (width, height))

# Process each TIFF file and write to the video
for tiff_file in tiff_files:
    frame = cv2.imread(os.path.join(input_dir, tiff_file))  # Read the TIFF file
    video_writer_avi.write(frame)  # Write the frame to the AVI video

# Release the VideoWriter object
video_writer_avi.release()

print(f"Video saved as {output_avi}")